In [6]:
!pip install boto3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
import boto3
import requests
import zipfile
import json
import io
import logging


In [8]:
%env AWS_ACCESS_KEY_ID = 
%env AWS_SECRET_ACCESS_KEY = 
%env AWS_DEFAULT_REGION = ap-south-1

env: AWS_ACCESS_KEY_ID=
env: AWS_SECRET_ACCESS_KEY=
env: AWS_DEFAULT_REGION=ap-south-1


In [ ]:
try:
  s3 = boto3.client(service_name = "s3",region_name = "ap-south-1")
  print("Sucessfully s3 client created")
except Exception as e:
      logging.error("Failed to create s3 client: %s", e, exc_info=True)
      print("Failed to create s3 client: ", e)

Sucessfully s3 client created


In [ ]:

from botocore.exceptions import ClientError


def create_bucket(bucket_name, region=None):
    """Create an S3 bucket in a specified region

    If a region is not specified, the bucket is created in the S3 default
    region (us-east-1).

    :param bucket_name: Bucket to create
    :param region: String region to create bucket in, e.g., 'us-west-2'
    :return: True if bucket created, else False
    """

    # Create bucket
    try:
        if region is None:
            s3_client = boto3.client('s3')
            s3_client.create_bucket(Bucket=bucket_name)
        else:
            s3_client = boto3.client('s3', region_name=region)
            location = {'LocationConstraint': region}
            s3_client.create_bucket(Bucket=bucket_name,
                                    CreateBucketConfiguration=location)
    except ClientError as e:
        logging.error(e)
        return False
    return True

In [ ]:
try:
    s3 = boto3.client('s3')
    response = s3.list_buckets()

    # Output the bucket names
    print('Existing buckets:')
    for bucket in response['Buckets']:
        print(f'  {bucket["Name"]}')
except ClientError as e:
  print(f'Error: {e}')

Existing buckets:
  submissionsproj1


In [ ]:
create_bucket("submissionsproj1",region = "ap-south-1")

True

In [ ]:
from io import BytesIO

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.93 Safari/537.36'}
url = "https://www.sec.gov/Archives/edgar/daily-index/bulkdata/submissions.zip"


# Download the zip file from the URL
response = requests.get(url,headers=headers)
content = response.content





In [ ]:
bucket_name = "submissionsproj1"
file_path = "datadump/Archive.zip"
try:
  with open(file_path, "rb") as file:
        content = file.read()
        with boto3.client("s3") as s3:
          s3.put_object(Bucket=bucket_name, Key=file_path, Body=content)
  print("Success")
except Exception as e:
  print(e)

Success


In [ ]:
try:
  bucket_name = "submissionsproj1"
  file_name = "datadump/Archive.zip"
  obj = s3.get_object(Bucket = bucket_name, Key = file_path)
  zip_file = io.BytesIO(obj["Body"].read())
  with zipfile.ZipFile(zip_file) as z:
    z.extractall()
  count= 0
  for file in z.namelist():
    count += 1
     
    s3.upload_file(Filename = file, Bucket = bucket_name ,Key = "silver/"+ file)
    if count >=50:
      break  
  print("success")
except Exception as e:
  print(e)


success


In [ ]:
import re

s3 = boto3.resource('s3')

bucket_name = 'submissionsproj1'

prefix = 'silver/'
pattern = r'^silver\/[A-Z]{3}\d{10}-submissions\-\d{3}\.json$'

bucket = s3.Bucket(bucket_name)

for obj in bucket.objects.filter(Prefix=prefix):
  if re.match(pattern,obj.key):
    print('Match found!')
    obj.delete()
  else:
    print('No match.')


In [ ]:

s3 = boto3.client('s3')
dynamodb = boto3.resource('dynamodb')

bucket_name = 'submissionsproj1'
prefix = 'silver/'

table_name = 'Proj1table'
table = dynamodb.Table(table_name)
try:

  response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)
  for obj in response.get('Contents', []):
      key = obj['Key']
      if key.endswith('.json'):
          # Read the contents of the file
          obj = s3.get_object(Bucket=bucket_name, Key=key)
          contents = obj['Body'].read().decode('utf-8')

          # Parse the JSON data
          data = json.loads(contents)
          table.put_item(Item=data)
except Exception as e:
    print(f'An error occurred: {e}')
